In [184]:
reset -fs

In [201]:
from boto.s3.connection import S3Connection
from ipyparallel import Client
import json


In [202]:

rc = Client()
engines = rc[:]
len(engines)

2

In [205]:
with engines.sync_imports():
    from boto.s3.connection import S3Connection
    import json


importing S3Connection from boto.s3.connection on engine(s)
importing json on engine(s)


In [246]:
## connect to aws
s3 = S3Connection()
bucket = s3.get_bucket('twitter-streaming02062017')
list_of_aws_keys = [str(i.name) for i in bucket.list()]

In [247]:
list_of_aws_keys


['twitter2017/02/08/17/twitter-streaming-data-1-2017-02-08-17-14-52-4fd9eb72-b2f7-45b3-a3ea-a5089f878e42',
 'twitter2017/02/08/17/twitter-streaming-data-1-2017-02-08-17-24-49-f282213f-6c58-46c6-86f8-ad5f5ad52201',
 'twitter2017/02/08/17/twitter-streaming-data-1-2017-02-08-17-29-47-2ae4da8f-f75a-40c8-a3ce-2e64d6a47f01',
 'twitter2017/02/08/22/twitter-streaming-data-1-2017-02-08-22-43-54-4c45b0ca-d655-4885-bf54-47c9826e5446',
 'twitter2017/02/08/22/twitter-streaming-data-1-2017-02-08-22-48-58-adfe836a-8bed-4bec-a2cb-33b5995d915a',
 'twitter2017/02/08/22/twitter-streaming-data-1-2017-02-08-22-54-13-0c666209-b2fc-49fd-bdab-2ee1d184d47c',
 'twitter2017/02/08/22/twitter-streaming-data-1-2017-02-08-22-59-23-510ef051-ffa1-4cb1-8ef3-ac0f596c57ee',
 'twitter2017/02/08/23/twitter-streaming-data-1-2017-02-08-23-04-19-06afc0f0-c6d6-4e0f-aa87-3d64ae2767ed',
 'twitter2017/02/08/23/twitter-streaming-data-1-2017-02-08-23-09-20-b8c54923-924c-4dc8-a3f8-578c38e3efa6',
 'twitter2017/02/08/23/twitter-stream

In [253]:
def twitter_scatter(individual_key):
    """Create a client to s3. Then, iterate through the keys in the given bucket. Return a counter of each hashtag."""
    s3 = S3Connection()
    bucket = s3.get_bucket('twitter-streaming02062017',validate=False)
    key_file = bucket.get_key(individual_key)
    strFile = key_file.get_contents_as_string()
    result = []
    count = 0
    for tweet in strFile.split('/n'):
        #print(type(tweet))
        try:
            entity = json.loads(tweet)
            entity = entity['entities']
            if entity != None and (entity['hashtags'] != None or len(entity['hashtags']) != 0):
                for hashtags in entity['hashtags']:
                    result.append(hashtags['text'])
        except ValueError:
            count+=1
    print(result)



In [245]:
str(list_of_aws_keys[0])

'twitter2017/02/08/17/twitter-streaming-data-1-2017-02-08-17-14-52-4fd9eb72-b2f7-45b3-a3ea-a5089f878e42'

In [211]:
twitter_scatter(list_of_aws_keys[0])

[u'\u0e25\u0e38\u0e07\u0e27\u0e34\u0e28\u0e27\u0e30', u'consumer', u'credit', u'HappyBirthdayHarry', u'ReplacementMigration', u'\u0627\u0644\u0627\u0642\u0632\u0627\u0645_\u062a\u062a\u0637\u0627\u0648\u0644_\u0639\u0644\u064a_\u0639\u0632\u0627\u0645', u'\u0645\u062c\u0627\u0647\u062f', u'\u0627\u0644\u0639\u0631\u0628\u064a\u0629', u'WetWednesday', u'Foibe', u'draintheswamp', u'VeteransForTrump', u'DailyGunDose', u'\u0639\u0627\u0644\u0645_\u0627\u0644\u062a\u0631\u0646\u062f', u'\u062e\u0627\u0644\u062f_\u0627\u0644\u0639\u062a\u064a\u0628\u064a_\u0641\u064a_\u0643\u0644\u0645\u062a\u064a\u0646', u'\u0647\u0627\u064a\u0643\u0646\u062c_\u0627\u0644\u0633\u0639\u0648\u062f\u064a\u0647', u'\u062f_\u0645\u0639\u0627\u0630_\u0627\u0633\u062f_\u062a\u0648\u064a\u062a\u0631_10k_\u0645\u062d\u0628', u'\uc628\ub77c\uc778\u3142\u314f\uce74\ub77c', u'\uc6d4\ub4dc\uc628\ub77c\uc778\u3142\u314f\uce74\ub77c\uac8c\uc784', u'\ud0c0\uc774\ud0c4\ubc14\ub450\uae30', u'\u3142\u314f\uce74\ub77c\ud558\ub

In [255]:
#print(client.ids)
print(len(list_of_keys))

## scatter the keys to each engine
engines.scatter('s3' , list_of_aws_keys).get()
%px y = [twitter_scatter(_) for _ in s3]


10


CompositeError: one or more exceptions from call to method: execute
[0:execute]: NameError: name 'twitter_scatter' is not defined
[1:execute]: NameError: name 'twitter_scatter' is not defined

In [249]:
list_of_aws_keys

['twitter2017/02/08/17/twitter-streaming-data-1-2017-02-08-17-14-52-4fd9eb72-b2f7-45b3-a3ea-a5089f878e42',
 'twitter2017/02/08/17/twitter-streaming-data-1-2017-02-08-17-24-49-f282213f-6c58-46c6-86f8-ad5f5ad52201',
 'twitter2017/02/08/17/twitter-streaming-data-1-2017-02-08-17-29-47-2ae4da8f-f75a-40c8-a3ce-2e64d6a47f01',
 'twitter2017/02/08/22/twitter-streaming-data-1-2017-02-08-22-43-54-4c45b0ca-d655-4885-bf54-47c9826e5446',
 'twitter2017/02/08/22/twitter-streaming-data-1-2017-02-08-22-48-58-adfe836a-8bed-4bec-a2cb-33b5995d915a',
 'twitter2017/02/08/22/twitter-streaming-data-1-2017-02-08-22-54-13-0c666209-b2fc-49fd-bdab-2ee1d184d47c',
 'twitter2017/02/08/22/twitter-streaming-data-1-2017-02-08-22-59-23-510ef051-ffa1-4cb1-8ef3-ac0f596c57ee',
 'twitter2017/02/08/23/twitter-streaming-data-1-2017-02-08-23-04-19-06afc0f0-c6d6-4e0f-aa87-3d64ae2767ed',
 'twitter2017/02/08/23/twitter-streaming-data-1-2017-02-08-23-09-20-b8c54923-924c-4dc8-a3f8-578c38e3efa6',
 'twitter2017/02/08/23/twitter-stream

In [250]:
engine_work = engines.apply(twitter_scatter,list_of_aws_keys)

In [251]:
engine_work.data

[{}, {}]

In [252]:
engine_data = engine_work.get()

CompositeError: one or more exceptions from call to method: twitter_scatter
[0:apply]: S3ResponseError: S3ResponseError: 400 Bad Request

[1:apply]: S3ResponseError: S3ResponseError: 400 Bad Request


In [ ]:
result = dview.gather('y')
from functools import reduce
product = reduce((lambda x, y: x + y ), result)

product.most_common()

In [28]:
%px hashtags = [twitter_scatter(i) for i in s3_keys ]

CompositeError: one or more exceptions from call to method: execute
[0:execute]: NameError: name 'twitter_scatter' is not defined
[1:execute]: NameError: name 'twitter_scatter' is not defined
[2:execute]: NameError: name 'twitter_scatter' is not defined
[3:execute]: NameError: name 'twitter_scatter' is not defined
.... 4 more exceptions ...

In [199]:
engines['s3_keys']

[[u'twitter2017/02/08/17/twitter-streaming-data-1-2017-02-08-17-14-52-4fd9eb72-b2f7-45b3-a3ea-a5089f878e42',
  u'twitter2017/02/08/17/twitter-streaming-data-1-2017-02-08-17-24-49-f282213f-6c58-46c6-86f8-ad5f5ad52201',
  u'twitter2017/02/08/17/twitter-streaming-data-1-2017-02-08-17-29-47-2ae4da8f-f75a-40c8-a3ce-2e64d6a47f01',
  u'twitter2017/02/08/22/twitter-streaming-data-1-2017-02-08-22-43-54-4c45b0ca-d655-4885-bf54-47c9826e5446',
  u'twitter2017/02/08/22/twitter-streaming-data-1-2017-02-08-22-48-58-adfe836a-8bed-4bec-a2cb-33b5995d915a'],
 [u'twitter2017/02/08/22/twitter-streaming-data-1-2017-02-08-22-54-13-0c666209-b2fc-49fd-bdab-2ee1d184d47c',
  u'twitter2017/02/08/22/twitter-streaming-data-1-2017-02-08-22-59-23-510ef051-ffa1-4cb1-8ef3-ac0f596c57ee',
  u'twitter2017/02/08/23/twitter-streaming-data-1-2017-02-08-23-04-19-06afc0f0-c6d6-4e0f-aa87-3d64ae2767ed',
  u'twitter2017/02/08/23/twitter-streaming-data-1-2017-02-08-23-09-20-b8c54923-924c-4dc8-a3f8-578c38e3efa6',
  u'twitter2017/02

In [ ]:
result = all_engines.gather('y')
from functools import reduce
product = reduce((lambda x, y: x + y ), result)

product.most_common()